---
# Tutoriel 1 - Familiarisation avec MNIST et un premier Neural Net avec PyTorch.
---

<center><img src="https://python.gel.ulaval.ca/media/sio-u009/mlprocess_3.png" alt="Processus d'apprentissage automatique" width="50%"/></center>

Dans ce tutoriel, nous allons reprendre l'exercice précédent sur les images de chiffres (MNIST) et reconstruire une réseau de neurones simple en utilisant [PyTorch](https://pytorch.org/) la librairie développée par Facebook pour faire des réseaux de neurons plus complexes que les simples [MLPs de Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html).

Voici les différentes étapes suivies par ce tutoriel:

1. Loading et exploration du dataset MNIST
2. Définition d'un réseau simple
3. Définition de l'optimiseur
4. Définition de la fonction de perte et d'une métrique
5. Boucle d'entraînement
    1. Piger une "minibatch" (pour la SGD)
    2. Forward Pass
    3. Back propagation
    4. Optimisation
6. Résultats sur l'ensemble de test

Commencons par charger les modules nécessaires et définir quelques fonctions :

In [ ]:
%matplotlib inline
import os
import math
import torch
import numpy as np
from torch import optim, nn
from torchvision.transforms import ToTensor
from torchvision.datasets.mnist import MNIST
from torch.utils.data import DataLoader, random_split

torch.manual_seed(42)
np.random.seed(42)

In [ ]:
def load_mnist(download=False, path='./', transform=None):
    """Loads the MNIST dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train and test dataset
    """
    train_dataset = MNIST(path, train=True, download=download, transform=transform)
    test_dataset = MNIST(path, train=False, download=download, transform=transform)
    return train_dataset, test_dataset


def load_mnist_with_validation_set(download=False, path='./', train_split=0.8):
    """Loads the MNIST dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train, valid and test dataset ready to be ingest in a neural network
    """
    train, test = load_mnist(download, path, transform=ToTensor())
    lengths = [round(train_split*len(train)), round((1.0-train_split)*len(train))]
    train, valid = random_split(train, lengths)
    return train, valid, test

def count_number_of_parameters(net):
    """ Count the number of parameters of a neural net

    :param net: a pytorch neural network
    :return: The number of parameters in the net
    """
    return sum(p.numel() for p in net.parameters() if p.requires_grad)

## 1. Loading et exploration du dataset MNIST

Analysons plus en détails notre dataset, nos inputs et nos outputs (ou nos x et y)

In [ ]:
train, test = load_mnist(download=True)

# Retirons les fichiers inutiles pour optimiser l'espace utilisé
!rm -f ./MNIST/raw/*ubyte*

In [ ]:
train, test

In [ ]:
len(train), len(test)

In [ ]:
first_image = train[0]
print(first_image)
first_image[0]

In [ ]:
?ToTensor

In [ ]:
tensor = ToTensor()(first_image[0])
tensor

In [ ]:
tensor.shape

In [ ]:
?train

In [ ]:
train.transform = ToTensor()
test.transform = ToTensor()

### Création d'un ensemble de validation

On va prendre 20% des données d'entrainement pour cela.

In [ ]:
lengths = [round(0.8*len(train)), round(0.2*len(train))]
train, valid = random_split(train, lengths)

In [ ]:
len(train), len(valid), len(test)

## 2. Définition d'un réseau simple

Quelques configurations

In [ ]:
use_gpu = torch.cuda.is_available()
n_epoch = 20
batch_size = 32
learning_rate = 0.01

Le tableau suivant est une représentation du réseau en couche. Le code ci-dessous utilise la manière orienté objet de PyTorch d'implémenter ce réseau.

| Type de couche              | Taille de sortie |      # de paramètres   |
|-----------------------------|:----------------:|:----------------------:|
| Input                       |   1x28x28   |              0            |
| Flatten                     |  1\*28\*28  |              0            |
| **Linear avec 10 neurones** |     10      | 28\*28\*10 + 10 = 7 850 |

\# total de paramètres du réseau: 7 850


In [ ]:
class Net(nn.Module): # La classe nn.Module représente un réseau de neurones dans PyTorch.
    def __init__(self):
        super(Net, self).__init__()
        # Nous instancions les couches du réseau dans la méthode __init__.
        self.fully_connected = nn.Linear(28 * 28, 10)

    def forward(self, x):
        # La méthode forward effectue les calculs faits par le réseau.
        x = x.view(-1, 28 * 28)
        x = self.fully_connected(x)
        return x

model = Net()
if use_gpu:
    model.cuda()
model

In [ ]:
parameters = list(model.parameters())
len(parameters)

In [ ]:
parameters[0], parameters[1]

In [ ]:
parameters[0].shape

In [ ]:
count_number_of_parameters(model)

## 3. Définition de l'optimiseur

L'optimiseur a besoin des paramètres du réseau à optimiser ainsi que du taux d'apprentissage (learning rate) pour connaître la grandeur des pas à faire pendant l'optimisation.

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

## 4. Définition de la fonction de perte et d'une métrique

NB: Dans PyTorch, la ```CrossEntropyLoss``` cache une ```Softmax```

In [ ]:
criterion = nn.CrossEntropyLoss()

Pour la métrique, nous allons utiliser l'exactitude fournir par ```sklearn```

In [ ]:
from sklearn.metrics import accuracy_score
def metric(y_true, y_pred): return accuracy_score(y_true, y_pred) * 100

## 5 Boucle d'entraînement

Nous définissons des "dataloaders" qui vont nous donner `batch_size` exemples à la fois, c'est-à-dire des batchs de taille `batch_size`. Nous avons 3 dataloaders pour les 3 ensembles utilisés: entraînement, validation et test.

In [ ]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

Nous définissons maintenant une classe Trainer qui aura comme tâche d'effectuer l'entraînement et l'évaluation de notre modèle. Dans le prochain tutoriel, nous verrons la librairie Poutyne qui va faire tout ça pour nous et bien plus.

Un des détail à comprendre à lisant le code ci-dessous est que le gradient est stocké dans les paramètres lorsque PyTorch le calcule pour nous. Nous n'avons donc pas besoin d'accéder au gradient directement. L'optimiseur se charge de tout pour nous.

In [ ]:
class Trainer:
    def __init__(self, model, optimizer, criterion, metric, use_gpu=False):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.metric = metric
        self.use_gpu = use_gpu
        
    def evaluate_model(self, loader):
        true = []
        pred = []
        val_loss = []

        # Activation du mode d'évaluation du modèle
        self.model.eval()
        
        # Désactivation du calcul du gradient en évaluation
        with torch.no_grad():
            for batch in loader:
                inputs, targets = batch
                
                # Envoie de la batch sur GPUs le cas échéant
                if self.use_gpu:
                    inputs = inputs.cuda()
                    targets = targets.cuda()

                # Ceci appelle ma méthode forward de notre modèle.
                output = self.model(inputs)

                predictions = output.max(dim=1)[1]

                val_loss.append(self.criterion(output, targets).item())
                true.extend(targets.detach().cpu().numpy().tolist())
                pred.extend(predictions.detach().cpu().numpy().tolist())

        return self.metric(true, pred), sum(val_loss) / len(val_loss)

    def train_model(self, train_loader, valid_loader, n_epoch):
        for i in range(n_epoch):
            # Activation du mode d'entraînement du modèle
            self.model.train()
            
            for batch in train_loader:
                inputs, targets = batch
                
                # Envoie de la batch sur GPUs le cas échéant
                if self.use_gpu:
                    inputs = inputs.cuda()
                    targets = targets.cuda()
                
                # Le gradient est mis à zéro pour éviter que le gradient de la batch
                # précédente soit mélangé au gradient de la nouvelle batch.
                self.optimizer.zero_grad()
                
                # Ceci appelle ma méthode forward de notre modèle.
                output = self.model(inputs)

                # Calcul de la perte
                loss = self.criterion(output, targets)
                
                # Retropropagation du gradient (le gradient est stocké dans les paramètres)
                loss.backward()
                
                # Mise à jour des poids en fonction du gradient calculé.
                self.optimizer.step()

            train_acc, train_loss = self.evaluate_model(train_loader)
            val_acc, val_loss = self.evaluate_model(valid_loader)
            
            print('Epoch {} - Train acc: {:.2f} - Val acc: {:.2f} - Train loss: {:.4f} - Val loss: {:.4f}'.format(
                i,
                train_acc,
                val_acc,
                train_loss,
                val_loss
            ))

C'est maintenant le temps d'entraîner et de tester notre modèle.

In [ ]:
trainer = Trainer(model, optimizer, criterion, metric, use_gpu)
trainer.train_model(train_loader, valid_loader, n_epoch)

In [ ]:
test_acc, test_loss = trainer.evaluate_model(test_loader)

In [ ]:
test_acc, test_loss